<a href="https://colab.research.google.com/github/ajayverma23/GenAI/blob/main/SemanticMatchingVertex_PaLM_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import auth as google_auth
google_auth.authenticate_user()

import vertexai
project_id = 'genai-testing-410703'; location = "us-central1"
vertexai.init(project=project_id, location=location)

In [5]:
import random

import pandas as pd
import vertexai
from vertexai.language_models import TextGenerationModel

pd.options.display.max_colwidth = 1000

vertexai.init()

In [12]:
amazon = pd.read_csv("/content/sample_data/Amazon.csv", encoding='unicode_escape')
amazon.columns = [
    "idAmazon",
    "amazon_title",
    "amazon_description",
    "amazon_manufacturer",
    "amazon_price",
]
amazon.head()

,idAmazon,amazon_title,amazon_description,amazon_manufacturer,amazon_price
0,b000jz4hqo,clickart 950 000 - premier image pack (dvd-rom),NaN,broderbund,0.00
1,b0006zf55o,ca international - arcserve lap/desktop oem 30pk,oem arcserve backup v11.1 win 30u for laptops and desktops,computer associates,0.00
2,b00004tkvy,noah's ark activity center (jewel case ages 3-8),NaN,victory multimedia,0.00
3,b000g80lqo,peachtree by sage premium accounting for nonprofits 2007,peachtree premium accounting for nonprofits 2007 is the affordable easy to use accounting solution that provides you with donor/grantor management. if you're like most nonprofit organizations you're constantly striving to maximize each and every dollar of your annual operating budget. financial reporting by programs and funds advanced operational reporting and the rock-solid core accounting features that have made peachtree the choice of hundreds of thousands of small businesses. the result is an accounting solution tailor-made for the challenges of operating a nonprofit organization. keep an audit trail to record and report on any changes made to your transactions improve data integrity with prior period locking archive your organization's data for snap shots of your data before you closed your year set up individual user profiles with password protection peachtree restore wizard restores all backed-up data files plus web transactions and customized forms includes all standard acc...,sage software,599.99
4,b0006se5bq,singing coach unlimited,singing coach unlimited - electronic learning products (win me nt 2000 xp),carry-a-tune technologies,99.99


In [13]:
google = pd.read_csv("/content/sample_data/GoogleProducts.csv",encoding='unicode_escape')
google.columns = [
    "idGoogleBase",
    "google_title",
    "google_description",
    "google_manufacturer",
    "google_price",
]
google.head()

,idGoogleBase,google_title,google_description,google_manufacturer,google_price
0,http://www.google.com/base/feeds/snippets/11125907881740407428,learning quickbooks 2007,learning quickbooks 2007,intuit,38.99
1,http://www.google.com/base/feeds/snippets/11538923464407758599,superstart! fun with reading & writing!,fun with reading & writing! is designed to help kids learn to read and write better through exercises puzzle-solving creative writing decoding and more!,NaN,8.49
2,http://www.google.com/base/feeds/snippets/11343515411965421256,qb pos 6.0 basic software,qb pos 6.0 basic retail mngmt software. for retailers who need basic inventory sales and customer tracking.,intuit,637.99
3,http://www.google.com/base/feeds/snippets/12049235575237146821,math missions: the amazing arcade adventure (grades 3-5),save spectacle city by disrupting randall underling's plan to drive all the stores out of business and take over the city. solve real-world math challenges in the uniquely entertaining stores and make theme successful again.,NaN,12.95
4,http://www.google.com/base/feeds/snippets/12244614697089679523,production prem cs3 mac upgrad,adobe cs3 production premium mac upgrade from production studio premium or standard,adobe software,805.99


In [14]:
matching = pd.read_csv("/content/sample_data/Amzon_GoogleProducts_perfectMapping.csv")
matching.head()

,idAmazon,idGoogleBase
0,b000jz4hqo,http://www.google.com/base/feeds/snippets/18441480711193821750
1,b00004tkvy,http://www.google.com/base/feeds/snippets/18441110047404795849
2,b000g80lqo,http://www.google.com/base/feeds/snippets/18441188461196475272
3,b0006se5bq,http://www.google.com/base/feeds/snippets/18428750969726461849
4,b00021xhzw,http://www.google.com/base/feeds/snippets/18430621475529168165


from matplotlib import pyplot as plt
import seaborn as sns
_df_0.groupby('idAmazon').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
_df_1.groupby('idGoogleBase').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
plt.subplots(figsize=(8, 8))
df_2dhist = pd.DataFrame({
    x_label: grp['idGoogleBase'].value_counts()
    for x_label, grp in _df_2.groupby('idAmazon')
})
sns.heatmap(df_2dhist, cmap='viridis')
plt.xlabel('idAmazon')
_ = plt.ylabel('idGoogleBase')

In [16]:
SAMPLE_SIZE = 60


def generate_test_and_eval_sets(sample_size=SAMPLE_SIZE):
    sample = matching.sample(sample_size)

    # Join the product information to the df of matching ID's
    matched_products = sample.merge(
        right=amazon, how="left", on="idAmazon"
    ).merge(right=google, how="left", on="idGoogleBase")

    google_descriptions = list(matched_products["google_title"])
    amazon_descriptions = list(matched_products["amazon_title"])

    # Create the dataframe of matching descriptions
    matching_descriptions = pd.DataFrame(
        {
            "description_1": google_descriptions,
            "description_2": amazon_descriptions,
            "match": "yes",
        }
    )

    # Create the dataframe of not matching descriptions
    amazon_descriptions_perm = [
        amazon_descriptions[i - 1] for i in range(len(amazon_descriptions))
    ]
    not_matching_descriptions = pd.DataFrame(
        {
            "description_1": google_descriptions,
            "description_2": amazon_descriptions_perm,
            "match": "no",
        }
    )

    full_dataset = pd.concat(
        [matching_descriptions, not_matching_descriptions], axis=0
    ).sample(len(matching_descriptions) * 2)

    evalset = full_dataset[:sample_size]
    testset = full_dataset[sample_size : 2 * sample_size]
    evalset.to_csv("/content/sample_data/Data/product_matching_eval.csv", index=None)
    testset.to_csv("/content/sample_data/Data/product_matching_test.csv", index=None)


# Uncomment to generate a different data sample
generate_test_and_eval_sets()

In [18]:
evalset = pd.read_csv("/content/sample_data/Data/product_matching_eval.csv")
testset = pd.read_csv("/content/sample_data/Data/product_matching_test.csv")

evalset.head()

,description_1,description_2,match
0,nova development corp ideasoft photo clip art 10000,pinnacle mobile media converter,no
1,final-draft final draft av 2.5 screenwriting software - mac/win screen writing software,crystal 3d impact! pro 1.25 (jewel case),no
2,global software a9716jcw - crossword addict - win 95 98 me 2000 xp,gsp crossword addict,yes
3,computer associates babwbr1151s33 - brightstor arcserve backup client for vss software snap-shot v.11.5 for windows with service pack 1 - add-on - backup & recovery - 1 server(s) - complete product - standard - multilingual - pc,ca arcserve bkup r11.5 win client vss software snap-shot- prod only,yes
4,global marketing partners antivirus v.10 - bit defender,adobe premiere pro cs3,no


In [19]:
evalset.match.value_counts()

no     30
yes    30
Name: match, dtype: int64

In [20]:
testset.match.value_counts()

yes    30
no     30
Name: match, dtype: int64

In [21]:
#Model implementation

generation_model = TextGenerationModel.from_pretrained("text-bison@001")

In [22]:
#outputs yes or no depending on whether the PaLM model thinks the product descriptions are matching.

def are_products_matching(d1, d2, prompt):
    prompt = prompt.format(desc1=d1, desc2=d2)
    answer = generation_model.predict(
        prompt, temperature=0.2, max_output_tokens=1024, top_k=40, top_p=0.8
    )
    return answer.text

In [23]:
#test on the evaluation set whether a given prompt seems to be working for this use case.

PROMPT = """
Are the following two descriptions describing the same product?
Answer only with "yes" or "no".

Description 1: {desc1}
Description 2: {desc2}
"""

index = random.randint(0, len(evalset) - 1)

d1 = evalset.iloc[index]["description_1"]
d2 = evalset.iloc[index]["description_2"]
ground_truth = evalset.iloc[index]["match"]
prediction = are_products_matching(d1, d2, prompt=PROMPT)


print(
    f"""
Are the following two descriptions describing the same product?

Description 1: {d1}

Descriptions 2: {d2}

MODEL ANSWER: {prediction}
GROUND TRUTH: {ground_truth}
"""
)


Are the following two descriptions describing the same product?

Description 1: kaspersky anti-virus 6.0 (pc)

Descriptions 2: kaspersky anti-virus 6.0

MODEL ANSWER: yes
GROUND TRUTH: yes



In [28]:
testset.shape

(60, 3)

In [29]:
#Scoring the test set

def apply_prompt(prompt, dataset):
    scored_dataset = dataset.copy()
    scored_dataset["predictions"] = scored_dataset.apply(
        lambda row: are_products_matching(
            row.description_1, row.description_2, prompt=PROMPT
        ),
        axis=1,
    )
    return scored_dataset

#apply this function to our testset
scored_testset = apply_prompt(prompt=PROMPT, dataset=testset)
scored_testset.to_csv("/content/sample_data/Data/scored_evalset.csv", index=False)

scored_testset.head()

ResourceExhausted: 429 Quota exceeded for aiplatform.googleapis.com/online_prediction_requests_per_base_model with base model: text-bison. Please submit a quota increase request. https://cloud.google.com/vertex-ai/docs/generative-ai/quotas-genai.